In [1]:
import os
import random
import numpy as np
import pandas as pd
from tqdm import tqdm

import torch
from torch.utils.data import Dataset

import evaluate
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import DataCollatorWithPadding
from transformers import TrainingArguments, Trainer

from sklearn.model_selection import train_test_split

SEED = 456
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
DEVICE

device(type='cuda')

In [3]:
BASE_DIR = os.getcwd()
DATA_DIR = os.path.join(BASE_DIR, 'data')
OUTPUT_DIR = os.path.join(DATA_DIR, 'outputs')

model_name = "klue/roberta-large"

# Load Datasets

In [5]:
original_path = os.path.join(DATA_DIR, 'train.csv')
original_df = pd.read_csv(original_path)
original_df.head()

,ID,text,target
0,ynat-v1_train_00000,정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보,4
1,ynat-v1_train_00001,K찰.국DLwo 로L3한N% 회장 2 T0&}송=,3
2,ynat-v1_train_00002,"m 김정) 자주통일 새,?r열1나가야1보",2
3,ynat-v1_train_00003,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩,5
4,ynat-v1_train_00004,pI美대선I앞두고 R2fr단 발] $비해 감시 강화,6


In [13]:
noised_path = os.path.join(DATA_DIR, 'kjs/noisy_df.csv')
noised_df = pd.read_csv(noised_path)
noised_df

,ID,text,target
0,ynat-v1_train_00000,정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보,4
1,ynat-v1_train_00001,K찰.국DLwo 로L3한N% 회장 2 T0&}송=,3
2,ynat-v1_train_00002,"m 김정) 자주통일 새,?r열1나가야1보",2
3,ynat-v1_train_00004,pI美대선I앞두고 R2fr단 발] $비해 감시 강화,6
4,ynat-v1_train_00006,프로야구~롯TKIAs광주 경기 y천취소,1
...,...,...,...
1594,ynat-v1_train_02787,13이 노바 라이n2·미J어패0 T3 10 결G상품% *,6
1595,ynat-v1_train_02788,남원소식 춘>X학%단+장Rn 모집,0
1596,ynat-v1_train_02789,"이총리,세4. H직 완fl지 못해E할 일 꽤;남아",2
1597,ynat-v1_train_02792,"높`X#E율…}BO Q""[/선수 몸값 상승 CAO",1


In [7]:
cleaned_path = os.path.join(DATA_DIR, 'fill_mask/train_generated.csv')
cleaned_df = pd.read_csv(cleaned_path)
cleaned_df.head()

,ID,text,target
0,ynat-v1_train_00000,정권이 파리 미사 참석에 이용기간 제한을 두고 분열을 보임,4
1,ynat-v1_train_00001,찰 국 로한 회장이 맡은 주요 프로젝트,3
2,ynat-v1_train_00002,"""김정은, 자주통일 새 열쇠를 잡고 나서야 한다""",2
3,ynat-v1_train_00003,"""갤럭시 노트 7, 주말 동안 10만 대 개통…시장 불법 보조금 논란에 얼룩져""",5
4,ynat-v1_train_00004,"""대선을 앞두고 감시 강화, 단발적 비해""",6


In [8]:
augmented_path = os.path.join(DATA_DIR, 'fill_mask/train_filled.csv')
augmented_df = pd.read_csv(augmented_path)
augmented_df

,ID,text,target
0,ynat-v1_train_00000,"한국 축구 대표팀, 월드컵 예선전서 3-1 승리",4
1,ynat-v1_train_00001,"중국, 코로나19 백신 개발 성공 및 안전성 검증 완료",3
2,ynat-v1_train_00002,"정부, 올해 경제 성장률 3% 예상 발표",2
3,ynat-v1_train_00003,"갤노트 주말 100만 대 개통, 시장은 불법 보조금 얼룩",5
4,ynat-v1_train_00004,대선 앞두고 1억 5천만 원 추가로 감시 강화,6
...,...,...,...
2795,ynat-v1_train_02795,트럼프 폭스뉴스 앵커들 충성도 점수 매겨 점만점에 50점,6
2796,ynat-v1_train_02796,삼성 갤럭시 S22 정식 출시 첫 주말 이통시장 침묵,2
2797,ynat-v1_train_02797,"텔레그램, 한 시간 동안 서비스 중단 및 정상화 과정",4
2798,ynat-v1_train_02798,"인터뷰 류현진, 친구에게 안타 맞는 것 싫어해 승부는 냉정한 것 같다",1


In [9]:
self_train_path = os.path.join(DATA_DIR, 'Self_train/self_train_step2.csv')
self_train_df = pd.read_csv(self_train_path)
self_train_df

,ID,text,target
0,ynat-v1_train_00000,KT 이용기간 2분 종료될 예정입니다.,4
1,ynat-v1_train_00001,K찰국대통령이 로L한 회장에게 2시간 동안 면담을 요청했다,3
2,ynat-v1_train_00002,"김정은, 자주통일 새해 메시지 발표",2
3,ynat-v1_train_00003,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩,4
4,ynat-v1_train_00004,미 대선 앞두고 R2F 단 발 비해 감시 강화,6
...,...,...,...
2795,ynat-v1_train_02795,트럼프 폭스뉴스 앵커들 충성도 점수매겨…10점만점에 12점도,6
2796,ynat-v1_train_02796,삼성 갤럭시S9 정식 출시 첫 주말 이통시장 잠잠,4
2797,ynat-v1_train_02797,"텔레그램, 중국 서버에서 2시간 동안 서비스 중단",4
2798,ynat-v1_train_02798,인터뷰 류현진 친구에게 안타 맞는 것 싫어해…승부는 냉정,1


# Make Train dataset

In [10]:
## 문자열 앞뒤 따옴표 제거
import re
def remove_outer_quotes(input_string):
    if type(input_string) == float :
        return ' '
    # 문자열의 앞뒤 공백을 제거
    cleaned = input_string.strip()
    
    # 작은따옴표와 큰따옴표 확인
    while cleaned.startswith('"') or cleaned.startswith("'"):
        cleaned = cleaned[1:]
    while cleaned.endswith('"') or cleaned.endswith("'"):
        cleaned = cleaned[:-1]
    
    return cleaned

# 테스트
example_string = '""코로나19 대면 수업, 원격 수업 대비 뚜렷한 선호도 드러나"""'
result = remove_outer_quotes(example_string)
print(result)

코로나19 대면 수업, 원격 수업 대비 뚜렷한 선호도 드러나


In [ ]:
## Cleaning noised data -> Label은 정상
denoising_df = self_train_df.copy()

for index, row in noised_df.iterrows():
    sample_id = row['ID']
    new_text = cleaned_df.loc[cleaned_df['ID'] == sample_id]['text'].values[0]
    new_text = remove_outer_quotes(new_text)
    
    if len(new_text) < 5 :
        denoising_df.drop(denoising_df[denoising_df['ID'] == sample_id].index, inplace=True)
    else :
        denoising_df.loc[denoising_df['ID'] == sample_id, 'text'] = new_text

denoising_df

,ID,text,target
0,ynat-v1_train_00000,정권이 파리 미사 참석에 이용기간 제한을 두고 분열을 보임,4
1,ynat-v1_train_00001,찰 국 로한 회장이 맡은 주요 프로젝트,3
2,ynat-v1_train_00002,"김정은, 자주통일 새 열쇠를 잡고 나서야 한다",2
3,ynat-v1_train_00003,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩,4
4,ynat-v1_train_00004,"대선을 앞두고 감시 강화, 단발적 비해",6
...,...,...,...
2795,ynat-v1_train_02795,트럼프 폭스뉴스 앵커들 충성도 점수매겨…10점만점에 12점도,6
2796,ynat-v1_train_02796,삼성 갤럭시S9 정식 출시 첫 주말 이통시장 잠잠,4
2797,ynat-v1_train_02797,"텔레그램 서버, 한 시간 동안 다운, 보안 우려 제기",4
2798,ynat-v1_train_02798,인터뷰 류현진 친구에게 안타 맞는 것 싫어해…승부는 냉정,1


In [15]:
## 데이터에서 불필요한 text row drop
denoising_df = denoising_df[denoising_df['text'].str.len() > 5]
denoising_df.to_csv(f'./data/Self_train/self_train_step2_cleaned.csv', index=False)

In [ ]:
## 미리 학습했던 model을 이용해 re-labeling (Step 2)
model = AutoModelForSequenceClassification.from_pretrained(OUTPUT_DIR+"/self_train_step1/checkpoint-350").to(DEVICE)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model.eval()

new_train = origin_train.copy()

for idx, sample in tqdm(origin_train.iterrows(), total=len(origin_train), desc="Evaluating"):
    target_id = sample['ID']

    if target_id in ids :
        continue

    inputs = tokenizer(sample['text'], return_tensors="pt").to(DEVICE)
    with torch.no_grad():
        logits = model(**inputs).logits
        pred = torch.argmax(torch.nn.Softmax(dim=1)(logits), dim=1).cpu().numpy()
        new_train.loc[new_train['ID'] == target_id, 'target'] = pred[0]

new_train.to_csv(f'./data/Self_train/self_train_step2.csv', index=False)


# DATA Augmentation with BM25

In [47]:
from rank_bm25 import BM25Okapi
import numpy as np
corpus = denoising_df['text'].values.tolist()
corpus = [sentence for sentence in corpus if len(sentence) > 5]
len(corpus)

2764

In [48]:
# Hugging Face tokenizer 로드 (예: BERT tokenizer)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenized_corpus = [tokenizer.tokenize(doc) for doc in corpus]

# BM25 모델 생성
bm25 = BM25Okapi(tokenized_corpus)


In [54]:
# augmentation된 문장들
augmented_corpus = np.concatenate((augmented_df['text'].values, cleaned_df['text'].values))
augmented_sentences = augmented_corpus.tolist()
augmented_sentences = [sentence for sentence in augmented_sentences if (type(sentence)!=float and len(sentence) > 5)]
print(len(augmented_sentences))
augmented_sentences

5514


[' 한국 축구 대표팀, 월드컵 예선전서 3-1 승리',
 ' 중국, 코로나19 백신 개발 성공 및 안전성 검증 완료',
 ' 정부, 올해 경제 성장률 3% 예상 발표',
 ' 갤노트 주말 100만 대 개통, 시장은 불법 보조금 얼룩',
 ' 대선 앞두고 1억 5천만 원 추가로 감시 강화',
 ' 정부, 코로나19 대유행에 대한 대응책 발표',
 ' 프로야구 롯데 자이언츠 광주 경기, 천만 관객을 위한 취소',
 ' 아가메즈, 득점 우리카드에 3골을 넣어 손해보험을 완파 위 굳',
 ' 삼성, 갤럭시 S21의 새로운 기능과 혁신: 5G, 화면, 카메라 기술 개선',
 ' 듀얼심 아이폰 하반기 출시설 솔솔, 알뜰폰',
 ' 삼성전자, 스마트폰 판매량 급증... (문맥에 따라 최종 복구 결과를 작성해주세요)',
 ' 정부, 개인소득세율 25% 인하 발표',
 ' 국내 코로나19 확진자 급증, 정부의 대응책 발표',
 ' 한국 경제 성장률 3.2%로 상승',
 ' 전국 고등학교, 대학수학능력시험 응시생 9만 명 넘어',
 ' 서울시, 3개 지역에 코로나19 백신 접종 시작',
 '  공사업체 협박 사건에 연루된 언론인이 분쟁 해결 명목으로 돈을 받고 집행유예 판결을 받다',
 ' 월세 전환에 따른 주거비 부담 증가, 작년 역대 최고치 2.5% 증가',
 ' 정부, 코로나19 백신 접종 계획 발표',
 '  페이스북의 인터넷 드론 아퀼라, 실물 첫 시험',
 ' 한국 경제, 세계 최대 경제 강국으로의 도약 - 우리나라 경제가 세계 최고 수준에 도달했다고 국제 경제 전문가들은 말합니다. ',
 ' 삼성, 갤럭시 S21 5G 출시 예정',
 '  현 차량 임원급 직원 채용 확대 계획 발표',
 ' 아시안게임에서 목소리를 높인 박항서 베트남, 일본을 이길 수 있을까?',
 ' 서울에 다시 오존주의보, 도심 서북동북권 발령 종합',
 ' "선 폭 현장 조사 결과 발표" ',
 ' 레콤, 분기 영업익 5천억원 증가, 사상 최대치 기록',
 '  생명보험 인증에 유럽의 대가들이

In [ ]:
# 유사도 계산 및 선택
threshold = 30
selected_sentences = []

for aug_sentence in tqdm(augmented_sentences):
    tokenized_query = tokenizer.tokenize(aug_sentence)
    scores = bm25.get_scores(tokenized_query)
    max_score = max(scores)

    if max_score < threshold:
        selected_sentences.append(aug_sentence)
        
# 선택된 문장들 추가
final_sentences = corpus + selected_sentences


100%|██████████| 5514/5514 [00:40<00:00, 135.15it/s]


In [99]:
len(final_sentences)

4121

In [104]:
## sentence list Dataframe으로 치환
data = {
    'ID': [f'ynat-v1_train_{i:05d}' for i in range(len(final_sentences))],
    'text': final_sentences,
    'target': [0] * len(final_sentences)
}

augmented_train_df = pd.DataFrame(data)
augmented_train_df

,ID,text,target
0,ynat-v1_train_00000,정권이 파리 미사 참석에 이용기간 제한을 두고 분열을 보임,0
1,ynat-v1_train_00001,찰 국 로한 회장이 맡은 주요 프로젝트,0
2,ynat-v1_train_00002,"김정은, 자주통일 새 열쇠를 잡고 나서야 한다",0
3,ynat-v1_train_00003,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩,0
4,ynat-v1_train_00004,"대선을 앞두고 감시 강화, 단발적 비해",0
...,...,...,...
4116,ynat-v1_train_04116,"설 맞이 도심 속 겨울 축제, 워커힐에서 즐기다",0
4117,ynat-v1_train_04117,"""인터넷은행 진출에 신중한 종합 은행, 분기 영업 이익에 힘입어""",0
4118,ynat-v1_train_04118,"""북한의 핵 실험 가능성, 언제쯤 김정은의 도발 징후가 나타날까?""",0
4119,ynat-v1_train_04119,"삼성, 올해 첫 대형 프로젝트 수주, 1조원 규모의 첨단 조선소 건설 계약 체결",0


In [107]:
model = AutoModelForSequenceClassification.from_pretrained("outputs/self_train_step1/checkpoint-350").to(DEVICE)
model.eval()

augmented_train_relabelled = augmented_train_df.copy()

for idx, sample in tqdm(augmented_train_df.iterrows(), total=len(augmented_train_df), desc="Evaluating"):
    target_id = sample['ID']
    
    inputs = tokenizer(sample['text'], return_tensors="pt").to(DEVICE)
    with torch.no_grad():
        logits = model(**inputs).logits
        pred = torch.argmax(torch.nn.Softmax(dim=1)(logits), dim=1).cpu().numpy()
        augmented_train_relabelled.loc[augmented_train_relabelled['ID'] == target_id, 'target'] = pred[0]

augmented_train_relabelled.to_csv(f'./data/Self_train/self_train_step3.csv', index=False)

Evaluating: 100%|██████████| 4121/4121 [01:32<00:00, 44.74it/s]


# Re-labelling 3

In [109]:
dataset_train, dataset_valid = train_test_split(augmented_train_relabelled, test_size=0.2, random_state=SEED)

class BERTDataset(Dataset):
    def __init__(self, data, tokenizer):
        input_texts = data['text']
        targets = data['target']
        self.inputs = []; self.labels = []
        for text, label in zip(input_texts, targets):
            tokenized_input = tokenizer(text, padding='max_length', truncation=True, return_tensors='pt')
            self.inputs.append(tokenized_input)
            self.labels.append(torch.tensor(label))

    def __getitem__(self, idx):
        return {
            'input_ids': self.inputs[idx]['input_ids'].squeeze(0),
            'attention_mask': self.inputs[idx]['attention_mask'].squeeze(0),
            'labels': self.labels[idx].squeeze(0)
        }

    def __len__(self):
        return len(self.labels)

data_train = BERTDataset(dataset_train, tokenizer)
data_valid = BERTDataset(dataset_valid, tokenizer)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

f1 = evaluate.load('f1')
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return f1.compute(predictions=predictions, references=labels, average='macro')

In [111]:
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR+'/Self_train_step3',
    overwrite_output_dir=True,
    do_train=True,
    do_eval=True,
    do_predict=True,
    report_to="wandb",
    logging_strategy='steps',
    eval_strategy='steps',
    save_strategy='steps',
    logging_steps=100,
    eval_steps=50,
    save_steps=50,
    save_total_limit=2,
    learning_rate= 2e-05,
    adam_beta1 = 0.9,
    adam_beta2 = 0.999,
    adam_epsilon=1e-08,
    weight_decay=0.01,
    lr_scheduler_type='linear',
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=2,
    num_train_epochs=10,
    load_best_model_at_end=True,
    metric_for_best_model='eval_f1',
    greater_is_better=True,
    seed=SEED
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=data_train,
    eval_dataset=data_valid,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [ ]:
trainer.train()

Step,Training Loss,Validation Loss,F1
50,No log,0.324325,0.888161
100,0.861300,0.344093,0.889226
150,0.861300,0.425726,0.875974
200,0.395400,0.400101,0.886577
250,0.395400,0.630774,0.863286
300,0.234400,0.507961,0.884126


KeyboardInterrupt: 

: 

In [5]:
model = AutoModelForSequenceClassification.from_pretrained("data/outputs/Self_train_step3/checkpoint-300").to(DEVICE)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model.eval()

st3_df = pd.read_csv(DATA_DIR + '/Self_train/self_train_step3.csv')
st3_df.head()


,ID,text,target
0,ynat-v1_train_00000,정권이 파리 미사 참석에 이용기간 제한을 두고 분열을 보임,6
1,ynat-v1_train_00001,찰 국 로한 회장이 맡은 주요 프로젝트,5
2,ynat-v1_train_00002,"김정은, 자주통일 새 열쇠를 잡고 나서야 한다",2
3,ynat-v1_train_00003,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩,4
4,ynat-v1_train_00004,"대선을 앞두고 감시 강화, 단발적 비해",2


In [ ]:
st3_relabelled = st3_df.copy()

for idx, sample in tqdm(st3_df.iterrows(), total=len(st3_df), desc="Evaluating"):
    target_id = sample['ID']
    
    inputs = tokenizer(sample['text'], return_tensors="pt").to(DEVICE)
    with torch.no_grad():
        logits = model(**inputs).logits
        pred = torch.argmax(torch.nn.Softmax(dim=1)(logits), dim=1).cpu().numpy()
        st3_relabelled.loc[st3_relabelled['ID'] == target_id, 'target'] = pred[0]

st3_relabelled.to_csv(f'./data/Self_train/self_train_step4.csv', index=False)

Evaluating: 100%|██████████| 4121/4121 [01:29<00:00, 45.98it/s]


: 